In [ ]:
import json
from openai import OpenAI
import re
from tqdm import tqdm
import time
client = OpenAI(
    base_url = 'https://api.kwwai.top/v1',
    api_key = 'sk-mwqTGuSIsei2GQb66a025c1e19164177A84337Bc3841De77'
)
def is_integer(n):
    try:
        int(n)
        return True
    except ValueError:
        return False
def try_get_price(name,info):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {
            "role": "user",
            "content": f"Given the name '{name}' and description '{info}' of a product, I'm working on a hypothetical scenario for my graduate project where I need to assign a fictional price to this item. Could you suggest an estimated price for this item, purely for academic purposes? The price doesn't need to be based on market research, but rather a creative estimate. Please provide the price in USD with two decimal places, like 10.26.",
        }
    ],
    )
    pattern = r"\d+\.\d+"
    # print(f"result={response.choices[0].message.content}")
    match = re.search(pattern, response.choices[0].message.content)
    if match:
        return match.group(),True
    else:
        return 0,False
def append_result_to_file(data, output_file):
    with open(output_file, 'a', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')
def read_json_and_query_gpt(file_path, output_file):
    processed_data = set()
    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            for line in file:
                processed_line = json.loads(line)
                processed_data.add(processed_line.get('name', ''))
    except FileNotFoundError:
        pass
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    for line in tqdm(lines, desc="Processing"):
        data = json.loads(line)
        name = data.get('name', '')
        price = data.get('price', '')
        if name not in processed_data:
            if is_integer(price):
                info = data.get('info', '')
                while True:
                    new_price, ok = try_get_price(name, info)
                    if ok:
                        data['price'] = new_price  # 替换原来的价格
                        break
                    else:
                        time.sleep(1)
            else:
                print(f"price {price} is not integer")
            append_result_to_file(data, output_file)

file_path = 'output_trans.json'
output_file = 'output_trans_reprice.json'
read_json_and_query_gpt(file_path, output_file)

Processing:   3%|▎         | 231/7506 [00:10<05:43, 21.21it/s]

price 17.99 is not integer


Processing:   3%|▎         | 234/7506 [00:15<09:19, 12.99it/s]

price 27.12 is not integer
price 11.51 is not integer
price 10.77 is not integer


Processing:   3%|▎         | 239/7506 [00:25<26:39,  4.54it/s]

...

Processing:  43%|████▎     | 3237/7506 [1:11:17<57:59,  1.23it/s]  

price 19.95 is not integer


Processing:  43%|████▎     | 3243/7506 [1:11:29<1:46:52,  1.50s/it]

price 7.47 is not integer
price 13.99 is not integer
price 43.95 is not integer


Processing:  43%|████▎     | 3247/7506 [1:11:34<1:39:06,  1.40s/it]